In [1]:
import numpy as np
import pandas as pd
import math as m
import cmath
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
from sympy import pprint
from scipy.optimize import minimize as GDlib
import MLE_functions as fn

rho:
⎡0.5   0   0⎤
⎢           ⎥
⎢ 0   0.5  0⎥
⎢           ⎥
⎣ 0    0   0⎦

 rho_num 
 [[ 0.4856+0.00000000e+00j -0.0006-6.23538291e-03j  0.0129-3.29089653e-03j]
 [-0.0006+6.23538291e-03j  0.517 +3.16574686e-18j -0.0035-1.81865335e-02j]
 [ 0.0129+3.29089653e-03j -0.0035+1.81865335e-02j -0.0026-1.84377578e-19j]]

 prob vec [0.0833333333, 0.0833333333, 0.1666666667, 0.0833333333, 0.0833333333, 0.1666666667, 0.0833333333, 0.0833333333, 0.1666666667]

 nj vec [869, 762, 1673, 904, 817, 1688, 799, 836, 1652]

 sum pj, sum nj  0.9999999999000001 10000


In [3]:
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')
c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2

# # analytic formuale for the probabilities, as given in the notes
pr1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
pr2 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
pr3 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
pr4 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
pr5= (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
pr7 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
pr8 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr9 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
pr = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]      # list of analytic prob.

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices


In [13]:
# GD main function
def GD_search_lib(N = 10000, params = [0,m.pi/2,0,0,m.pi/4] , guess_params = [.1, .3, 0, 0, m.pi/4], show_calcs = False):
    bounds = [(-m.pi,m.pi+.1), (-m.pi,m.pi+.1) ,(-m.pi,m.pi+.1),(-m.pi,m.pi+.1),(0,m.pi/2+.1)]
    num_expt = fn.num_experiment(N = N, params = params)
    states, rho, th_pr = num_expt[0], num_expt[1], num_expt[2] # for later
    nj_vec = num_expt[3]
    global pr                                           # importing the Analytical probabilities
    lnL_list = [nj_vec[i]*sym.log(pr[i]) for i in range(len(pr))]     
    lnL_abst = sum(lnL_list).evalf()                                  
    def lnL(sp = guess_params):                                    #s_p stands for search parameters
        lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1] , phi1: sp[2] , phi2: sp[3] , alpha: sp[4] })
        neg_likelihood = -(lnL.evalf())
        return neg_likelihood 
    #grad function
    def grad(p=[0,m.pi/2,0,0,m.pi/4] , s=(1/500), show_calcs = False):    # "point" is the list of parameters
        del_theta1 = lnL(sp = [p[0]+s,p[1],p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_theta2 = lnL(sp = [p[0],p[1]+s,p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi1   = lnL(sp = [p[0],p[1],p[2]+s,p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi2   = lnL(sp = [p[0],p[1],p[2],p[3]+s,p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_alpha  = lnL(sp = [p[0],p[1],p[2],p[3],p[4]+s]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])

        grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
        grad = [i if abs(i) > 1e-5 else 0 for i in grad]
        return grad
    def GD(p0 = [0,m.pi/2,0,0,m.pi/4], nit = 20, step = 1/120, show_calc = False):
        cost0 = lnL(p0)
        s = step
        grad_new = grad(p = p0)
        params_new = p0
        values = []
        for i in range(nit):
            grad_mod = (sum([abs(i)**2 for i in grad_new]))**(.5)
            params_new = [params_new[i]-s*(grad_new[i]/grad_mod) for i in range(len(p0))]
            grad_new = grad(params_new)
            value = [lnL(params_new), params_new, grad_new, i]
            values.append(value)
            if i > (3/10)*nit//1 and i > 10:
                s = 5*step
                if i > (8/10)*nit//1:
                    s = 15*step
            if show_calc == True:
                print(value,'\n ')
        return params_new

    # GDresult = GDlib(lnL, params0, bounds = bounds)
    opt = GDlib(lnL, guess_params, bounds = bounds)
    opt_params = opt.x
    
    # now to find the Fidelity
    opt_states_raw = fn.Creating_states(opt_params)[0]              #new states
    
    fid0_sym, fid1_sym = (abs(opt_states_raw[0].H*states[0])**2).evalf(), (abs(opt_states_raw[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid0_cross_sym, fid1_cross_sym = (abs(opt_states_raw[1].H*states[0])**2).evalf(), (abs(opt_states_raw[0].H*states[1])**2).evalf()
    fid0_cross, fid1_cross = complex(fid0_cross_sym[0]).real, complex(fid1_cross_sym[0]).real

    flip_algo = 0
    if fid0_cross + fid1_cross > fid0+fid1:                     # adding zero because of type being matrix..
        opt_params_1 = [opt_params[1],opt_params[0],opt_params[2],opt_params[3],opt_params[4]]        # reversing oder of theta1 and theta2 if cross fidelities are higher.
        flip_algo= 1
        # delta = abs(opt_params_1[0]-opt_params_1[1])/4                                                  # creating a buffer for new bounds
        # bound2 = [(i-delta, i+delta) for i in opt_params_1]                                             # buffer of size delta around each parameter
        # opt_2 = GDlib(lnL, opt_params_1, bounds = bound2)
        # opt_params_2 = opt_2.x
        # opt_params= opt_params_2
        opt_params_2 = GD(opt_params_1)
        opt_params= opt_params_2

    opt_states = fn.Creating_states(opt_params)[0]         #new correctly switched states
    fid0_sym, fid1_sym = (abs(opt_states[0].H*states[0])**2).evalf(), (abs(opt_states[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid = [fid0, fid1]

    if show_calcs == True:
        # pprint(rho)
        # print("collapse vector",nj_vec, "\ngiven parameters\n", params, "\noptimal parameters\n", opt_params)
        # print('\noriginal states: \n' )
        # pprint(states)
        print('\n raw states:' )
        print(list(opt_states_raw[0]))
        print(list(opt_states_raw[1]))
        print('\n final states:')
        print(list(opt_states[0]))
        print(list(opt_states[1]))
        print("\n raw params ", opt.x)
        if flip_algo == 1:
            print("\n intermediate params ", opt_params_1)
        print("\n final params ", opt_params)
    
        if flip_algo == 1:
            print("\n Fidelities raw", [fid0_cross, fid1_cross])
        print("\n Fidelities final",fid)
        
    return [opt_params,opt_states,fid]


In [5]:
# generalizing creating states function to complex states
params00 = [0,m.pi/2,0,0,m.pi/4]
params00complex = [0,m.pi/2,0,m.pi/2,m.pi/4]

a = (fn.Creating_states(params00complex)[2])
pprint(a)
#fucntion verified 

⎡0.5             0             0⎤
⎢                               ⎥
⎢          6.28318530717959⋅ⅈ   ⎥
⎢ 0   0.5⋅ℯ                    0⎥
⎢                               ⎥
⎣ 0              0             0⎦


In [6]:
# Generalizing num expt funciton to Complex states

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix

def num_experiment(N = 10000, params = [0,m.pi/2,0,0,m.pi/4], show_calcs = False, seed = None):
    if seed is not None:
        rand.seed(seed)
    
    creation = fn.Creating_states( params = params, Abstract = 0) # theoretical rho #@@@
    states = creation[0]
    rho = creation[-1]
    prob_vec_sympy =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)]    # created list of Th. probabilities
    prob_vec_raw = [(float(i.as_real_imag()[0])+float(i.as_real_imag()[1])*1j) for i in prob_vec_sympy]  # this is to avoid error, to convert sympy float to ordinary number
    prob_vec = [round(i.real, 10) for i in prob_vec_raw if abs(i.imag) < .0001]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials
    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     

    r_vec = np.dot(M_inv,pj_num_vec)
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    # Loop for reconstructing the numerical matrix
    for i in rho_num_list:
        rho_num = np.add(rho_num, i)

    if show_calcs == True: 
        print('rho:')
        pprint(rho)
        print("\n rho_num \n", rho_num)
        print("\n prob vec", prob_vec)
        print("\n nj vec",nj_vec)
        print("\n sum pj, sum nj ", sum(prob_vec), sum(nj_vec))
    
    return [states, rho, rho_num, prob_vec, nj_vec]

# res = num_experiment(N = 10000, params= params00complex, show_calcs = 1)
# res[0]
# res[1]
# res[2]
# res[3]
# res[4]
# fucntion verified

In [7]:
# generalizing inversion function to complex states
# Fidelity Function
def Inversion( N = 10000, params = [0,m.pi/2, m.pi/4], threshold = 'variable'):    # coeffs are as list [theta_1, theta_2, alpha] in radians
    experiment = fn.num_experiment(N= N, params = params, show_calcs= False)
    rho = experiment[1]
    rho_num = experiment[2]                                                # !!! can enhance using postprocessing here (2*rho_num = rho_num+ conj(rho_num)). can also do directly in num_expt funciton
    eigenvalues, eigenvectors = np.linalg.eig(rho_num)
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .0001])  # !!! does this rounding affect things later..?
    evecs = np.around(eigenvectors, decimals=9)
    min_eval = np.min([abs(i) for i in evals])
    def solve_quadratic(a, b, c):
        d = (b**2) - (4*a*c)
        sol1 = (-b-np.sqrt(d))/(2*a)
        sol2 = (-b+np.sqrt(d))/(2*a)
        return [sol1, sol2]
    if len(params) ==3:                                 # For studying the simpler case with phis = 0
        a0, a1 = np.cos(params[0]), np.sin(params[0])
        b0, b1 = np.cos(params[1]), np.sin(params[1])
    elif len(params) == 5:
        a0, a1 = np.cos(params[0]), np.exp(params[2]*1j)*np.sin(params[0])
        b0, b1 = np.cos(params[1]), np.exp(params[3]*1j)*np.sin(params[1])
    c = [a0, a1, b0, b1]                                # four list of state coefficients

    if threshold != 'variable':
        if type(threshold) is str:
            print("Error! Threshold should be a number ")
    else:
        threshold = (4.2)/(N**.5)           # variable threshold


    
    xi = evecs[:, np.argmin([abs(i) for i in evals])]     # xi vector is same as the perp vector 
    c00,c01,c11 = xi[0],xi[2],xi[1]                         # defining In this weird way coz new basis arrangement, last elt is the |+> basis therefore c01 = xi[2]
    # the reconstructing the states' coefficients if-else loops
    c_raw = 0
    if abs(c00) > threshold and abs(c11) > threshold: 
        # print("condition 1")              
        if abs(c11)/abs(c00) < 2:
            soln  =  solve_quadratic(c00, np.sqrt(2)*c01, c11)
            # soln  =  solve_quadratic(c00, 2*c01, c11)
            z_a,z_b = soln[0], soln[1]
            a1 = (1/(np.sqrt(1+abs(z_a)**2)))           
            a0 = np.conj(z_a)*a1
            b1 = (1/(np.sqrt(1+abs(z_b)**2)))
            b0 = np.conj(z_b)*b1
            c_raw = [a0,a1,b0,b1]
            # if abs(phi1) > 90:          #
            #     phi1 = 180 - phi1
            # if abs(phi2) > 90:
            #     phi2 = 180 - phi2

            # phi1_mod, phi2_mod = 180-abs(angles[0]-angles[1]), 180-abs(angles[2]-angles[3])
            

        else:
            soln  =  solve_quadratic(c11, np.sqrt(2)*c01, c00)
            # soln  =  solve_quadratic(c11, 2*c01, c00)
            z_a,z_b = soln[0], soln[1]
            a0 = (1/(np.sqrt(1+abs(z_a)**2)))                          
            a1 = np.conj(z_a)*a0
            b0 = (1/(np.sqrt(1+abs(z_b)**2)))
            b1 = np.conj(z_b)*b0
            c_raw = [a0,a1,b0,b1]
            # print(z_a,z_b)
            # print('coeffs',a0,a1,b0,b1)
    elif abs(c00) <= threshold and abs(c11) > threshold:      # some cases are excluded if we don't use eqality
        a0, a1 = 1,0        # random choice, matching later
        # z = np.conj((-ca settrecer11)/(2*c01))     # z defined as (a0*/a1*)
        z = np.conj((-c11)/(np.sqrt(2)*c01))     # z defined as (a0*/a1*)
        b1= (1/(np.sqrt(1+abs(z)**2))) 
        b0= np.conj(z)*b1
        phi1, phi2 = np.angle(a0), np.angle(b0)                 # angle phi0 taken to be zero. coz for states where one of the coefficient is zero, even the relative phase doesnt change fidelity.See notebook
        print('phis #3:', phi1, phi2 )
        # print('condition #2')
        c_raw = [a0,a1,b0,b1]
    elif abs(c00) > threshold and abs(c11) <= threshold:
        a0, a1 = 0,1        
        # z = np.conj((-2*c01)/(c00))      # z defined as (b0*/b1*)
        z = np.conj((-(np.sqrt(2))*c01)/(c00))      # z defined as (b0*/b1*)
        b1= (1/(np.sqrt(1+abs(z)**2))) 
        b0= np.conj(z)*b1
        # print('condition #3, z = ',z)
        c_raw = [a0,a1,b0,b1]
        phi1, phi2 = np.angle(a1), np.angle(b0)
        print('phis #4:', phi1, phi2 )
    elif abs(c00) <= threshold and abs(c11) <= threshold:
        a0, a1 = 0,1        
        b0, b1 = 1,0     
        c_raw = [a0,a1,b0,b1]
        phi1, phi2 = np.angle(a1), np.angle(b0)
        print('phis #5:', phi1, phi2 )

    f00,f01 = (abs((c_raw[0]*c[0]+c_raw[1]*c[1])))**2, (abs((c_raw[0]*c[2]+c_raw[1]*c[3])))**2
    f10,f11 = (abs((c_raw[2]*c[0]+c_raw[3]*c[1])))**2, (abs((c_raw[2]*c[2]+c_raw[3]*c[3])))**2

    if  (f00 + f11) > (f01+ f10) :       
        c_num = c_raw
    else:
        c_num = [c_raw[2],c_raw[3],c_raw[0],c_raw[1]]

    F_a = (abs(sum([np.conj(c[i])*c_num[i] for i in range(0,2)])))**2
    F_b = (abs(sum([np.conj(c[i])*c_num[i] for i in range(2,4)])))**2
    F = [F_a, F_b]


    # finding final parameters from the final coeffs
    tht1 = (180/m.pi)*np.arccos(abs(c_num[0]))      # magnutude coz by construction any phase is pushed into the global phase (which also changes the relative phase)
    tht2 = (180/m.pi)*np.arccos(abs(c_num[2]))
    # print('theta 1 before and after')         # because it flips to the next quadrant by the exact amount that it is far from the 180 degree line because cosine is symmetric
    # print(tht1)
    # if c_num[1] < 0 :
    #     tht1 = 180+ (180-tht1)      
    # print(tht1)
    # print('theta 2 before and after')
    # print(tht2)
    # if c_num[3] < 0:
    #     tht2 = 180+ (180-tht2)
    # print(tht2)


    #finding the phi angles
    angles = np.angle(c_num)
    angles = (180/m.pi)*angles  #in degrees
    phi_corrections = [180 if 90 < i < 180 or 270 < i < 360 else 0 for i in [tht1, tht2]]
    phis = [(angles[0]-angles[1])-phi_corrections[0], (angles[2]-angles[3])-phi_corrections[1]]
    phi1, phi2 = phis[0], phis[1]
    
    #finding the priors, see notebook
    psi0_num, psi1_num = [c_num[0], c_num[1]], [c_num[2], c_num[3]]
    psi0psi0_num = np.array([psi0_num[0]*psi0_num[0], psi0_num[1]*psi0_num[1], (np.sqrt(2)*psi0_num[0]*psi0_num[1])], dtype = complex) 
    psi1psi1_num = np.array([psi1_num[0]*psi1_num[0], psi1_num[1]*psi1_num[1], (np.sqrt(2)*psi1_num[0]*psi1_num[1])], dtype = complex)              # @@@ the square rooting is simplistic. see notebook for better. (cross term last due to prof's basis arrangement)
    overlap_num = np.dot(np.conjugate(psi0psi0_num),psi1psi1_num)
    psi1_cross_psi1_num = np.outer(psi1psi1_num, np.conjugate(psi1psi1_num))
    prior0 = (np.dot(np.dot(np.conjugate(psi0psi0_num), (rho_num - psi1_cross_psi1_num)), psi0psi0_num))/(1-abs(overlap_num)**2)        # see derivation in the notebook
    alpha = np.arccos(prior0)
    alpha_degrees = alpha*(180/m.pi)
  
    psi0_cross_psi0_num = np.outer(psi0psi0_num, np.conjugate(psi0psi0_num))
    prior1 = (np.dot(np.dot(np.conjugate(psi1psi1_num), (rho_num - psi0_cross_psi0_num)), psi1psi1_num))/(1-abs(overlap_num)**2)        # see derivation in the notebook
    alpha_1 = np.arccos(prior1)
    alpha_degrees_1 = alpha_1*(180/m.pi)
    
    final_params = [tht1, tht2, phi1, phi2, alpha_degrees]         

    if len(params) == 3:
        print('\n initial params in degrees= ', [i*(180/m.pi) for i in [params[0],params[1],0,0,params[2]]])           # if three parameters r given, phi are zero trivially
    else:
        print('initial params = ', [np.around(i,5) for i in params])
    print('final params = ', [np.around(i,5) for i in final_params])

    return [F, params ,final_params]


In [10]:
#RoughPad: testing the quality of output parameters for the inversion code
# Inversion(params = [m.pi/4,m.pi/2+m.pi/4, m.pi/4])      # eg.
# Inversion(params = [0,m.pi/2, m.pi/4])
# Inversion(params = [0,m.pi/4, m.pi/4])
# Inversion(params = [0,m.pi/8, m.pi/4])
# Inversion(params = [m.pi/2,m.pi/2+m.pi/4, m.pi/4])
# Inversion(params = [m.pi/2+m.pi/4,m.pi-.1, m.pi/4])
Inversion(params = [m.pi,m.pi+m.pi/6, m.pi/4])

phis #3: 0.0 -0.09998367321825048

 initial params in degrees=  [180.0, 210.0, 0.0, 0.0, 45.0]
final params =  [0.0, 24.83324, 0.0, -5.72864, (70.21946-0j)]


[[1.0, 0.9902416254647235],
 [3.141592653589793, 3.665191429188092, 0.7853981633974483],
 [0.0,
  24.833235583302425,
  0.0,
  -5.728642495620954,
  (70.21946027959973-1.2079052956430666e-16j)]]

NameError: name 'sym_probs' is not defined